### Hyper-parameter Tunning with Genetic Algorithm
#### Created: Dec. 29 2018

In [1]:
from pprint import pprint
import datetime
import os

In [2]:
import sys
sys.path.append("../")
# If this notebook file is not placed under in /notebook/ directory,
# adding directory "../" might not correly add the project directory.
# If adding "../" does not solve the importing problem, we need to setup 
# the directory mannually.
try:
    import constants
except ModuleNotFoundError:
    core_dir = input("Directory of core files >>> ")
    if not core_dir.endswith("/"):
        core_dir += "/"
    sys.path.append(core_dir)
    import constants

In [7]:
import core.training.genetic_hpt as genetic_hpt
from core.ga.genetic_hpt import GeneticHPT
import core.models.stacked_lstm as stacked_lstm

In [4]:
# Load dataset.
print("Dataset paths saved:")
pprint(constants.DATA_DIR)
choice = None
while choice is None or choice not in constants.DATA_DIR.keys():
    if choice is not None:
        print("Invalid data location received, try again...")
    choice = input("Select Dataset >>> ")
FILE_DIR = constants.DATA_DIR[choice]
print(f"Dataset chosen: {FILE_DIR}")

Dataset paths saved:
{'a': '/Users/tianyudu/Documents/Academics/EconForecasting/AnnEconForecast/data/UNRATE.csv',
 'b': '/home/ec2-user/AnnEconForecast/data/UNRATE.csv',
 'c': '/home/ec2-user/AnnEconForecast/data/DEXCAUS.csv'}
Select Dataset >>> a
Dataset chosen: /Users/tianyudu/Documents/Academics/EconForecasting/AnnEconForecast/data/UNRATE.csv


In [5]:
# Load configuration.
print("Avaiable configuration files found: ")
for cf in os.listdir("../hps_configs"):
    if cf.endswith("config.py"):
        print("\t" + cf)

config_name = input("Select config file >>> ")
if config_name.endswith(".py"):
    config_name = config_name[:-3]
# config_name = "mac_config"

exec(f"import hps_configs.{config_name} as config")

Avaiable configuration files found: 
	ec2_config.py
	mac_config.py
Select config file >>> mac_config


In [6]:
# Training Spec.
GENE_POOL = config.main
TOTAL_GEN = 30
POP_SIZE = 10
IGNORE_SET = (
    "PERIODS", "ORDER", "TRAIN_RATIO", "VAL_RATIO", "num_outputs", "num_inputs", "report_periods",
    "tensorboard_path", "model_path", "fig_path"
)

In [8]:
def eval_net_wrapper(param):
    return genetic_hpt.eval_net(
        model=stacked_lstm.StackedLSTM,
        param=param,
        file_dir=FILE_DIR,
        metric="mse_val",
        smooth_metric=0.05
    )

In [10]:
# Create the Optimizer.
optimizer = GeneticHPT(
    gene_pool=config.main,
    pop_size=POP_SIZE,
    eval_func=eval_net_wrapper,
    mode="min",
    retain=0.5,
    shot_prob=0.05,
    mutate_prob=0.05,
    verbose=False,
    ignore=IGNORE_SET
)

In [11]:
start_time = datetime.datetime.now()

In [12]:
genetic_hpt.train_op(
    optimizer=optimizer,
    total_gen=TOTAL_GEN,
    elite=5
)

Generation: [0/30]
Evaluating the initial population.
 ##------------------ Evaluating population[1/10,  10.00%]

KeyboardInterrupt: 

In [ ]:
end_time = datetime.datetime.now()
print(f"Time taken: {end_time - start_time}")